In [47]:
import pprint
import orcid
from requests import RequestException

In [48]:
api = orcid.PublicAPI()

In [49]:
ringgoldid= '6187'
place = "Oklahoma"

In [50]:
university = api.search_public('%s %s' % (ringgoldid, place), rows=10)

In [51]:
university

{'error-desc': None,
 'message-version': '1.2',
 'orcid-profile': None,
 'orcid-search-results': {'num-found': 1607,
  'orcid-search-result': [{'orcid-profile': {'client-type': None,
     'group-type': None,
     'orcid': None,
     'orcid-activities': None,
     'orcid-bio': {'biography': None,
      'contact-details': None,
      'delegation': None,
      'external-identifiers': None,
      'keywords': None,
      'personal-details': {'credit-name': None,
       'family-name': {'value': 'Brewster'},
       'given-names': {'value': 'Keith'},
       'other-names': None},
      'researcher-urls': None,
      'scope': None},
     'orcid-deprecated': None,
     'orcid-history': None,
     'orcid-id': None,
     'orcid-identifier': {'host': 'orcid.org',
      'path': '0000-0002-6614-8564',
      'uri': 'http://orcid.org/0000-0002-6614-8564',
      'value': None},
     'orcid-internal': None,
     'orcid-preferences': None,
     'type': None},
    'relevancy-score': {'value': 0.94465363}},


In [52]:
ringgoldid= '6187'
place='University of Oklahoma'

In [53]:
results = uni.get('orcid-search-results').get('orcid-search-result')

In [54]:
result = results[0]

In [55]:
result

{'orcid-profile': {'client-type': None,
  'group-type': None,
  'orcid': None,
  'orcid-activities': None,
  'orcid-bio': {'biography': None,
   'contact-details': None,
   'delegation': None,
   'external-identifiers': None,
   'keywords': None,
   'personal-details': {'credit-name': None,
    'family-name': {'value': 'Brewster'},
    'given-names': {'value': 'Keith'},
    'other-names': None},
   'researcher-urls': None,
   'scope': None},
  'orcid-deprecated': None,
  'orcid-history': None,
  'orcid-id': None,
  'orcid-identifier': {'host': 'orcid.org',
   'path': '0000-0002-6614-8564',
   'uri': 'http://orcid.org/0000-0002-6614-8564',
   'value': None},
  'orcid-internal': None,
  'orcid-preferences': None,
  'type': None},
 'relevancy-score': {'value': 0.94472635}}

In [56]:
theid = result.get('orcid-profile').get('orcid-identifier').get('path')
print (theid)
fname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('given-names').get('value')
print (fname)
lname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('family-name').get('value')
print (lname)

0000-0002-6614-8564
Keith
Brewster


In [57]:
fname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('given-names').get('value')
print (fname)

Keith


In [58]:
type(fname)

str

In [59]:
for key in result:
    print (key)


orcid-profile
relevancy-score


In [60]:
import pprint
pprint.pprint(result.keys())

dict_keys(['orcid-profile', 'relevancy-score'])


In [61]:
r = (result['orcid-profile'])

In [62]:
pprint.pprint(r.keys())

dict_keys(['orcid-deprecated', 'orcid-id', 'group-type', 'orcid-activities', 'orcid-identifier', 'type', 'orcid-preferences', 'orcid-history', 'orcid-bio', 'orcid', 'orcid-internal', 'client-type'])


In [63]:
theid = '0000-0001-5544-0976'
summary = api.read_record_public(theid, 'activities')

In [64]:
summary

{'educations': {'education-summary': [{'created-date': {'value': 1392755678486},
    'department-name': 'Physics and Astronomy ',
    'end-date': {'day': None, 'month': None, 'year': {'value': '1982'}},
    'last-modified-date': {'value': 1392761508155},
    'organization': {'address': {'city': 'Lexington',
      'country': 'US',
      'region': 'KY'},
     'disambiguated-organization': {'disambiguated-organization-identifier': '4530',
      'disambiguation-source': 'RINGGOLD'},
     'name': 'University of Kentucky'},
    'path': '/0000-0001-5544-0976/education/123811',
    'put-code': 123811,
    'role-title': 'BS in Physics',
    'source': {'source-client-id': None,
     'source-name': {'value': 'Mark Laufersweiler'},
     'source-orcid': {'host': 'orcid.org',
      'path': '0000-0001-5544-0976',
      'uri': 'http://orcid.org/0000-0001-5544-0976'}},
    'start-date': {'day': None, 'month': None, 'year': {'value': '1978'}},
    'visibility': 'PUBLIC'},
   {'created-date': {'value': 1

In [65]:
employ = summary.get('employments').get('employment-summary')

In [66]:
pprint.pprint(employ)

[{'created-date': {'value': 1414175023832},
  'department-name': 'Department of Meteorology',
  'end-date': {'day': None,
               'month': {'value': '06'},
               'year': {'value': '1999'}},
  'last-modified-date': {'value': 1414175023832},
  'organization': {'address': {'city': 'Tallahassee',
                               'country': 'US',
                               'region': 'FL'},
                   'disambiguated-organization': {'disambiguated-organization-identifier': '7823',
                                                  'disambiguation-source': 'RINGGOLD'},
                   'name': 'Florida State University'},
  'path': '/0000-0001-5544-0976/employment/540305',
  'put-code': 540305,
  'role-title': 'Computer Research Specialist',
  'source': {'source-client-id': None,
             'source-name': {'value': 'Mark Laufersweiler'},
             'source-orcid': {'host': 'orcid.org',
                              'path': '0000-0001-5544-0976',
                 

In [67]:
id.setdefault(theid,[]).append(fname + ' ' + lname)
dict.setdefault('name',[]).append(id)

In [123]:
import pprint
import orcid
import time
from requests import RequestException

def search_persons(ringgoldid, place):
    api = orcid.PublicAPI()
    persons = {}
    who = {}
    counter = 1
    person_counter = 0
    row = 20
    maxsearch = 5
    
    for step in range(0, maxsearch, row):
        print (step, row)
        try:
            university = api.search_public('%s %s' % (ringgoldid, place), start=str(step), rows=row)
            for result in university.get('orcid-search-results').get('orcid-search-result'):
                orcid_id = result.get('orcid-profile').get('orcid-identifier').get('path')
                fname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('given-names').get('value')
                lname = result.get('orcid-profile').get('orcid-bio').get('personal-details').get('family-name').get('value')
                who.setdefault(orcid_id, []).append(fname + ' ' + lname )
                print (orcid_id +  " " + fname + ' ' + lname)
                try:
                    summary = api.read_record_public(orcid_id, 'activities')
                    try:
                        for employment in summary.get('employments').get('employment-summary'):
                            if employment.get('organization').get('disambiguated-organization').get('disambiguated-organization-identifier') == ringgoldid:
                                if not employment.get('end-date'):
                                    
                                    persons.setdefault(orcid_id, {}).setdefault((fname + ' ' + lname), []).append(employment.get('department-name'))
                                    
                                    #persons.setdefault(employment.get('department-name'), {}).setdefault(orcid_id, []).append(fname + ' ' + lname)
                                    #persons.setdefault(employment.get('department-name'), []).append(orcid_id)
                                    person_counter += 1
                    except AttributeError:
                        pass
                except RequestException as e:
                    print (e.response.text)
                print ('Processing record %d' % counter)
                counter += 1
            # print (person_counter)
            # pprint.pprint(persons)
        except RequestException as e:
            print (e.response.text)
        time.sleep(3)
        return persons

    
    print (person_counter)
    pprint.pprint(persons)

# search_persons('9142', 'Bochum')
# search_persons('14311', 'Dortmund')


In [125]:
tester = search_persons ('6187', 'University of Oklahoma')

0 20
0000-0002-6614-8564 Keith Brewster
Processing record 1
0000-0001-9835-113X Sean Arms
Processing record 2
0000-0003-0703-3801 Eric Johnson
Processing record 3
0000-0002-2173-8851 Andrew Kierig
Processing record 4
0000-0001-7715-2836 Elizabeth Knapp
Processing record 5
0000-0001-7791-9750 Tao Zhao
Processing record 6
0000-0002-8066-7643 Derrell Cox II
Processing record 7
0000-0003-4424-4923 Sylwester Ratowt
Processing record 8
0000-0002-9505-9069 Bruce Hoagland
Processing record 9
0000-0002-8323-4894 Daniel Curtis
Processing record 10
0000-0001-6599-6054 Leslie Sadler
Processing record 11
0000-0002-5418-121X Meta G Carstarphen
Processing record 12
0000-0002-3313-6284 William Taylor
Processing record 13
0000-0002-6857-919X Rachel Jones-Summan
Processing record 14
0000-0003-4969-5600 Jessica Fesler
Processing record 15
0000-0002-1998-3532 Tara Malone
Processing record 16
0000-0002-1453-3866 Tyler Haas
Processing record 17
0000-0003-1566-542X Logan Cox
Processing record 18
0000-0002-45

In [127]:
len (tester)

9

In [108]:
dict = {}
dict.setdefault('a', {}).setdefault('b', []).append('mark')

In [109]:
dict

{'a': {'b': ['mark']}}